In [1]:
import torch
from models import SkipConnectionSupervenientFeatureNetwork
from trainers import train_feature_network

lt.monkey_patch()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

seed = 44
torch.manual_seed(seed)



In [2]:
config = {
    "torch_seed": seed,
    "dataset_type": "ecog",
    "num_atoms": 64,
    "batch_size": 1000,
    "train_mode": True,
    "train_model_B": False,
    "adjust_Psi": False,
    "clip": 5,
    "feature_size": 3,
    "epochs": 70,
    "start_updating_f_after": 500,
    "update_f_every_N_steps": 5,
    "minimize_neg_terms_until": 9999999999,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 512, 512],
        "hidden_sizes_encoder_2": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-3,
    }
}


In [3]:
dataset = ECoGDataset()

trainloader = torch.utils.data.DataLoader(dataset, batch_size=config["batch_size"], shuffle=True)

In [17]:
# bits_dataset_config = {
#     "num_data_points": int(1e6),
#     "extra_bit_correlation": 0.99,
#     "parity_bit_correlation": 0.99,
# }


# dataset = BitStringDataset(
#     gamma_parity=bits_dataset_config['parity_bit_correlation'],
#     gamma_extra=bits_dataset_config['extra_bit_correlation'],
#     length=bits_dataset_config['num_data_points'],
# )

# trainloader = torch.utils.data.DataLoader(
#     dataset,
#     batch_size=config['batch_size'],
#     shuffle=True,
# )

# print("Dataset loaded")

Dataset loaded


In [4]:


import importlib
import trainers
importlib.reload(trainers)
from trainers import train_feature_network

feature_network = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)

feature_network = train_feature_network(config, trainloader, feature_network)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dmcsharry. Use `wandb login --relogin` to force relogin


Training: 100%|██████████| 70/70 [1:16:01<00:00, 65.16s/it]


In [30]:

config = {
    "torch_seed": seed,
    "dataset_type": "ecog",
    "num_atoms": 64,
    "batch_size": 1000,
    "train_mode": True,
    "train_model_B": True,
    "adjust_Psi": False,
    "clip": 5,
    "feature_size": 3,
    "epochs": 70,
    "start_updating_f_after": 500,
    "update_f_every_N_steps": 5,
    "minimize_neg_terms_until": 9999999999,
    "downward_critics_config": {
        "hidden_sizes_v_critic": [512, 1024, 1024, 512],
        "hidden_sizes_xi_critic": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "decoupled_critic_config": {
        "hidden_sizes_encoder_1": [512, 512, 512],
        "hidden_sizes_encoder_2": [512, 512, 512],
        "critic_output_size": 32,
        "lr": 1e-3,
        "bias": True,
        "weight_decay": 0,
    },
    "feature_network_config": {
        "hidden_sizes": [256, 256, 256, 256, 256],
        "lr": 1e-4,
        "bias": True,
        "weight_decay": 1e-3,
    }
}

import importlib
import trainers
importlib.reload(trainers)
from trainers import train_feature_network

feature_network_A = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
    ).to(device)

model_A_path = '/vol/bitbucket/dm2223/info-theory-experiments/models/ecog_feature_network_robust-star-3.pth'
feature_network_A.load_state_dict(torch.load(model_A_path))

feature_network_B = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)

feature_network_B = train_feature_network(config, trainloader, feature_network_B, feature_network_A)


<<Training model B>>


Training: 100%|██████████| 70/70 [1:11:27<00:00, 61.25s/it]


In [35]:
from einops import reduce
import numpy as np

feature_network_A = SkipConnectionSupervenientFeatureNetwork(
    num_atoms=config['num_atoms'],
    feature_size=config['feature_size'],
    hidden_sizes=config['feature_network_config']['hidden_sizes'],
    include_bias=config['feature_network_config']['bias'],
).to(device)

model_A_path = '/vol/bitbucket/dm2223/info-theory-experiments/models/feature_network_apricot-pond-70.pth'


feature_network_A.load_state_dict(torch.load(model_A_path))

feature_network = feature_network_A

# plot the various values of V to visually inspect if they contain information about the different bits


binary_numbers = torch.tensor([[int(bit) for bit in f"{i:06b}"] for i in range(2**6)])

v = feature_network(binary_numbers.float().to(device))

xor_bits = (reduce(binary_numbers[:, :5], 'b n -> b', 'sum') % 2).unsqueeze(1)

# plot the different values of v on a histogram, with different colors for the two different xor bits

import matplotlib.pyplot as plt
import numpy as np

v_np = v.detach().cpu().numpy()
xor_bits_np = xor_bits.cpu().numpy()

plt.hist(v_np[xor_bits_np == 0], bins=100, alpha=0.5, label='xor_bit=0')
plt.hist(v_np[xor_bits_np == 1], bins=100, alpha=0.5, label='xor_bit=1')
plt.legend()
plt.show()


extra_bits = binary_numbers[:, -1].unsqueeze(1)

# plot the different values of v on a histogram, with different colors for the two different extra bits


plt.hist(v_np[extra_bits == 0], bins=100, alpha=0.5, label='extra_bit=0')
plt.hist(v_np[extra_bits == 1], bins=100, alpha=0.5, label='extra_bit=1')
plt.legend()
plt.show()


bonus_bits = ( xor_bits + extra_bits ) % 2

# plot the different values of v on a histogram, with different colors for the two different bonus bits


plt.hist(v_np[bonus_bits == 0], bins=100, alpha=0.5, label='bonus_bit=0')
plt.hist(v_np[bonus_bits == 1], bins=100, alpha=0.5, label='bonus_bit=1')
plt.legend()
plt.show()



RuntimeError: Error(s) in loading state_dict for SkipConnectionSupervenientFeatureNetwork:
	Missing key(s) in state_dict: "hidden_layers.3.weight", "hidden_layers.3.bias", "hidden_layers.4.weight", "hidden_layers.4.bias". 
	size mismatch for initial_projection.weight: copying a param with shape torch.Size([256, 6]) from checkpoint, the shape in current model is torch.Size([256, 64]).
	size mismatch for final_projection.weight: copying a param with shape torch.Size([1, 256]) from checkpoint, the shape in current model is torch.Size([3, 256]).
	size mismatch for final_projection.bias: copying a param with shape torch.Size([1]) from checkpoint, the shape in current model is torch.Size([3]).